# Random Acts of Pizza

![Pizza Badges](https://storage.googleapis.com/kaggle-competitions/kaggle/3949/media/pizzas.png)

---

**Auteur**: Akim van Eersel  
**Date**: 2022-12-30

## Predire la donation de pizza à travers un subreddit dédié

### Subreddit RAOP

Le r/RAOP permet d'obtenir des données sur des demandes faites par des internautes, afin de potentiellement se voir offrir une pizza.  
Ces données se rapportent à la demande *(contenu et métadonnées)* ainsi qu'à certaines informations du profil Reddit du demandeur.

### Hypothèses

**Localisation** : d'un point de vue business, l'utilisation de données provenant de Reddit ne doit pas influer sur la population ciblée.  
**Décalage temporel** : les données datent d'il y a 9 ans et se sont écoulées sur une période de 3 ans. Le fait de faire une donation et qu'il s'aggisse d'une pizza est jugé socialement et économiquement intemporel.  
**Sagesse des foules** : la donation d'un-e individu à un-e autre est déterminée comme étant impartialement légitime *(petit delta dû au procédé global)*.

### Enjeux business

Pouvoir prédire la donation de pizza est projeté sur une étude de cas, où une chaîne de pizzeria cherche à mettre en place un procédé similaire, pour gagner entre autres en notoriété de marque.  
L'enjeu est d'obtenir un modèle prédisant la légitimité d'une demande par un-e individu au moment de celle-ci.

#### Décision applicative

Ce modèle ne devrait pas être l'unique décisionnaire de donation, surtout si ce dernier n'est pas ultra-performant.  
Celui-ci devrait être réutilisé en entrée d'un second algorithme de décision, face à d'autres variables liées aux implications business *(supply chain, etc)*.

In [1]:
import string

import numpy as np
import pandas as pd
from datetime import datetime as dt
from scipy.stats import f_oneway
import joblib

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.feature_selection import mutual_info_classif, chi2, SelectKBest
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import xgboost as xgb

from nltk.corpus import stopwords
from nltk import word_tokenize

### Présentation des données

In [2]:
pizza_raw_data = pd.read_json('../data/pizza_data.json',
                              dtype={"giver_username_if_known": str,
                                     "number_of_upvotes_of_request_at_retrieval": int,
                                     "post_was_edited": bool,
                                     "request_id": str,
                                     "request_number_of_comments_at_retrieval": int,
                                     "request_text": str,
                                     "request_text_edit_aware": str,
                                     "request_title": str,
                                     "requester_account_age_in_days_at_request": float,
                                     "requester_account_age_in_days_at_retrieval": float,
                                     "requester_days_since_first_post_on_raop_at_request": float,
                                     "requester_days_since_first_post_on_raop_at_retrieval": float,
                                     "requester_number_of_comments_at_request": int,
                                     "requester_number_of_comments_at_retrieval": int,
                                     "requester_number_of_comments_in_raop_at_request": int,
                                     "requester_number_of_comments_in_raop_at_retrieval": int,
                                     "requester_number_of_posts_at_request": int,
                                     "requester_number_of_posts_at_retrieval": int,
                                     "requester_number_of_posts_on_raop_at_request": int,
                                     "requester_number_of_posts_on_raop_at_retrieval": int,
                                     "requester_number_of_subreddits_at_request": int,
                                     "requester_received_pizza": bool,
                                     "requester_subreddits_at_request": list,
                                     "requester_upvotes_minus_downvotes_at_request": int,
                                     "requester_upvotes_minus_downvotes_at_retrieval": int,
                                     "requester_upvotes_plus_downvotes_at_request": int,
                                     "requester_upvotes_plus_downvotes_at_retrieval": int,
                                     "requester_user_flair": str,
                                     "requester_username": str,
                                     "unix_timestamp_of_request": int,
                                     "unix_timestamp_of_request_utc": int})

pizza_prevented_data = pizza_raw_data.loc[:, ~(pizza_raw_data
                                               .columns
                                               .isin(["giver_username_if_known",
                                                      "requester_user_flair",
                                                      "request_text",
                                                      "post_was_edited"]))
                       ]

dataset_period = (dt.strptime("29/09/2013", "%d/%m/%Y") - dt.strptime("08/12/2010", "%d/%m/%Y")).days

target_name = 'requester_received_pizza'
seed = 101
X = pizza_prevented_data.copy()
y = X.pop(target_name)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=seed, stratify=y_test)


univariate_features = ["unix_timestamp_of_request_utc", #non-utc timestamp is redundant and less convenient
                       "request_title",
                       "request_text_edit_aware"]

at_request_features = []
at_retrieval_features = []

for selected_time, selected_features in {"at_request": at_request_features, "at_retrieval": at_retrieval_features}.items():
    dataset_features = (X_train
                        .filter(regex=f'.*{selected_time}$')
                        .columns
                        .tolist())
    selected_features.extend(dataset_features)

pizza_retrieval_data = X_train[univariate_features + at_retrieval_features]
X_train = X_train[univariate_features + at_request_features]

wrapup_data_prez = f'''Le dataset est composé de {pizza_raw_data.shape[0]} observations, l'entrainement est fait sur 90% de l'ensemble, à savoir {X_train.shape[0]} observations.
Parmi les {len(y)} requêtes, {y.sum()} ont mené à une donation, ce qui représente {round(y.sum()/len(y), 3)*100}%.
Parmi la période de {dataset_period} jours, 1 pizza a été donné tous les {round(dataset_period/y_train.sum(), 2)} jour.

Au sein de ce dataset, il existe {pizza_prevented_data.shape[1]} variables, dont {pizza_prevented_data.select_dtypes(exclude=['object']).shape[1]} non textuelles et {pizza_prevented_data.select_dtypes(include=['object']).shape[1]} textuelles.
Pour la modélisation seulement {len(univariate_features + at_request_features)} de ces variables seront utilisées.'''

In [3]:
print(wrapup_data_prez)

Le dataset est composé de 4040 observations, l'entrainement est fait sur 90% de l'ensemble, à savoir 3636 observations.
Parmi les 4040 requêtes, 994 ont mené à une donations, ce qui représente 24.6%.
Parmi la période de 1026 jours, 1 pizza a été donné tous les 1.15 jour.

Au sein de ce dataset, il existe 28 variables, dont 23 non textuelles et 5 textuelles.
Pour la modélisation seulement 13 de ces variables seront utilisées.


In [4]:
def add_relative_votes(df, num, den, suffix="request"):
    math_fct = lambda row: row.iloc[1] and row.iloc[0] / row.iloc[1] or 0
    df[f'requester_relative_consensual_votes_at_{suffix}'] = df.copy()[[num, den]].apply(math_fct, axis = 1)

    return df

diff_votes = 'requester_upvotes_minus_downvotes_at_request'
sum_votes = 'requester_upvotes_plus_downvotes_at_request'

def add_text_length(df):
    post_feat = 'request_text_edit_aware'
    df[[f'{post_feat}_len']] = df.copy()[[post_feat]].apply(len)

    return df

X_test_lotr = add_text_length(X_test)
X_test_lotr = add_relative_votes(X_test_lotr, num=diff_votes, den=sum_votes)

xgb_lotr_model = joblib.load(f'../models/xgb_nlp_f1_pipeline.joblib')

def print_pipe_results(data_set, model):
    X_set, y_set = data_set
    yhat = model.predict(X_set)
    print(f'Out-of-samples classification report:\n\n {classification_report(y_set, yhat)}')
    print()
    print(f'''Confusion matrix:
{pd.DataFrame(confusion_matrix(y_set, yhat, normalize='all'),
              columns=["PredNeg", "PredPos"],
              index=["Neg", "Pos"])}''')

## Modelisation

### Métrique d'évaluation

La precision est la métrique d'évaluation choisie, en raison de son implication d'un point de vue business.  
En effet, celle-ci cherchant à diminuer les faux positifs, on se prémunit ainsi des demandes labellisées comme étant légitimes mais erronées, évitant de procéder aux dons impactant l'aspect financier.  

Toute chose étant égale par ailleurs, il est nécessaire de garder un oeil sur le nombre de faux négatifs afin d'avoir une certaine balance acceptable si possible.

### Modèle

Le modèle retenu fait partie de la famille des ensembles d'arbres de décision, une des versions les plus évolués, à savoir xgboost.

Pour tirer au mieux des capacités de ce modèle, une étape de preprocessing est nécessaire dans le cas de données textuelles.
Cette dernière se compose d'un workflow usuel en NLP :
1. Création d'un BOW, incluant les 2-grams, avec un tokenizer de mots provenant de la librairie NLTK, en gardant uniquement les comptes de 4 ou plus.
2. Transformation de cette matrice de décompte en une matrice TF-IDF pour obtenir les occurrences les plus pertinentes.
3. Réduction dimensionnelle des 100 composantes les plus importantes selon la factorisation NMF.

Cette modélisation permet ainsi d'obtenir les résultats suivants sur des données de test :

In [5]:
print_pipe_results([X_test, y_test], xgb_lotr_model)

Out-of-samples classification report:

               precision    recall  f1-score   support

       False       0.77      0.89      0.83       152
        True       0.36      0.18      0.24        50

    accuracy                           0.72       202
   macro avg       0.56      0.54      0.53       202
weighted avg       0.67      0.72      0.68       202


Confusion matrix:
      PredNeg   PredPos
Neg  0.673267  0.079208
Pos  0.202970  0.044554


### Performances

Les performances observées ne sont pas très satisfaisantes, mais sont malgré tout à mettre en perspective vis-à-vis de l'objectif visé et des données en entrées.  
En effet, comme le démontre le nombre de vrais et faux négatifs et des 72% d'accuracy, le modèle possède une aisance pour traiter les demandes dites illégitimes.  

Là où le bât blesse est lorsqu'il s'agit de bien traiter les vrais et faux positifs.  
En effet, les 18% de recall entrainent de ce fait l'algorithme à labelliser la plupart des demandes légitimes en tant qu'illégitimes.  

Par rapport, à l'objectif business, il est quand même préférable d'avoir cette situation que celle inverse.

In [6]:
pred_donation_pct = round(xgb_lotr_model.predict(X_test_lotr).sum() / len(xgb_lotr_model.predict(X_test_lotr)), 3) * 100
real_donation_pct = round(y.sum()/len(y), 3)*100

wrapup_concl_prez = f'''Sur les {round(y.sum()/len(y), 3)*100}% de requêtes réelles ayant donné lieu à une donation, le modèle actuel prédit {pred_donation_pct}% de requêtes légitimes devrant donner lieu à une donation.
Cela représente une différence de {round(pred_donation_pct/real_donation_pct, 3)*100}%.'''

## Conclusion

Les données non textuelles et textuelles travaillent de concert, avec leur poids relatif, pour permettre d'améliorer l'algorithme.  
Il serait préjudiciable de se passer d'un des deux types.   
Toutefois, les données textuelles sont au cœur de la modélisation.  

Après de nombreux essais, xgboost s'est avéré être un bon candidat en le combinant avec une étape de preprocessing pertinente.  
Cependant, l'algorithme est relativement strict sur les faux négatifs et conduit à des performances de recall +/- insuffisantes.

In [7]:
print(wrapup_concl_prez)

Sur les 24.6% de requêtes réelles ayant donné lieu à une donation, le modèle actuel prédit 12.4% de requêtes légitimes devrant donner lieu à une donation.
Cela représente une différence de 50.4%.
